<font size=7>Energy Exchange Simulator</font>

# Importing essential modules

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import xlwt
import xlsxwriter
from tabulate import tabulate

# GAME OPTIONS

In [ ]:
#####################
### BASIC OPTIONS ###
#####################

### World Size ###
size_map = 100            #>>>>> size of world map [Default = 100]

### AI Info ###
NDummy_source = 100       #>>>>> number of AI sellers participating in the market [Default = 100]
NDummy_sink = 100         #>>>>> number of AI buyers participating in the market [Default = 100]

### Day Intializer and Doomsday ###
day = 1                   #>>>>> Initializing Day [Default = 1]
dday = 5                  #>>>>> Doomsday [Default = 5]

### Player Initial Fund (€) ###
pfund = 5000000         #>>>>> Initial Player Fund

### 1 - use numerical value, 0 - use world data points ### for Renewable Capacity for the day 
choice1 = 0

### 1 - saves data to computer, 0 - does not save to computer ###
choice2 = 0


########################
### ADVANCED OPTIONS ### >>>Changing these option may result in Errors/No MCP,MCV interesection point<<<
########################

AI_source_var = 50         #>>>>> variance of AI seller capacity [Default = 50]
AI_sink_var = 50           #>>>>> variance of AI buyer demand [Default = 1]

AI_sell_bid_prof = [50,75] #>>>>> profit range for AI sellers [Default = [50,75]]
AI_buy_bid_impor = 100     #>>>>> demand factor for AI buyer (if set at 100 - AI will be willing to pay upto 100% extra price) [Default = 100]

### This option applies only when you use numerical values for Renewable Capacity for the day (i.e.) choice1 = 1 ###
hydro_prob = 30            #>>>>> uncertanity of Hydro Power Plants (if value is 30 : (70% to 100%) of max capcity will be generated) [Default = 30] 
wind_prob = 50             #>>>>> uncertanity of Wind Power Plants [Default = 50]
solar_prob = 70            #>>>>> uncertanity of Solar Power Plants [Default = 70]

# Power Plant Data

In [ ]:
PPtyp = []
PPpw = []
PPcap = []
PPco2_emis = []
PPdeg_coeff = []
PPxcoord = []
PPycoord = []

### CO2 production per ton/MW ###
co2_coal_prod = 23.868
co2_CCGT_prod = 9.936

##### POWER PLANT DATA ########################
PPtyp.append('Nuclear')
PPpw.append(500)
PPcap.append(2500000)
PPco2_emis.append(0) # ton/MW per day
PPdeg_coeff.append(0.05)
PPxcoord.append(random.randrange(0,size_map))
PPycoord.append(random.randrange(0,size_map))

PPtyp.append('Coal')
PPpw.append(250)
PPcap.append(1000000)
PPco2_emis.append(co2_coal_prod) # ton/MW per day
PPdeg_coeff.append(0.02)
PPxcoord.append(random.randrange(0,size_map))
PPycoord.append(random.randrange(0,size_map))

PPtyp.append('CCGT')
PPpw.append(200)
PPcap.append(1000000)
PPco2_emis.append(co2_CCGT_prod) # ton/MW per day
PPdeg_coeff.append(0.02)
PPxcoord.append(random.randrange(0,size_map))
PPycoord.append(random.randrange(0,size_map))

PPtyp.append('Hydro')
PPpw.append(120)
PPcap.append(1000000)
PPco2_emis.append(0) # ton/MW per day
PPdeg_coeff.append(0.025)
PPxcoord.append(random.randrange(0,size_map))
PPycoord.append(random.randrange(0,size_map))

PPtyp.append('Wind')
PPpw.append(80)
PPcap.append(500000)
PPco2_emis.append(0) # ton/MW per day
PPdeg_coeff.append(0.025)
PPxcoord.append(random.randrange(0,size_map))
PPycoord.append(random.randrange(0,size_map))

PPtyp.append('Solar')
PPpw.append(50)
PPcap.append(500000)
PPco2_emis.append(0) # ton/MW per day
PPdeg_coeff.append(0.025)
PPxcoord.append(random.randrange(0,size_map))
PPycoord.append(random.randrange(0,size_map))

PPdata = [PPtyp,PPpw,PPcap,PPco2_emis,PPdeg_coeff,PPxcoord,PPycoord]
###############################################

# Class definitions for Source(**Generator**) and Sink(**Consumer**)

In [ ]:
class Source:
    
    """Represent a power plant, with a name"""
    population = 0
    total_location = []
    
    def __init__(self, name, source, power, gen_cost, cap_cost, co2_emission, co2_cost, location=[]):
        
        self.name = name # type+random number(+(AI) if dummy)
        self.source_type = source  # Nuclear, Coal, CCGT, Hydro, wind, solar
        self.power = power   # [MW] Max Power Production Capacity
        self.power_day = power # [MW] Power Prodcution Capacity for the day (Applies only for Renewable Sources)
        self.gen_cost = gen_cost   # (Fuel cost) [€/MW]
        self.cap_cost = cap_cost   # [€] ### Buying price of power plant (will remain constant)
        self.cur_val = cap_cost    # [€] ### Current Value of power plant (will decrease with use)
        self.co2_emission = co2_emission ### [ton/MW] 
        self.co2_cost = co2_cost # [€/ton]
        self.location = location   # [Co-ordinates for the location on the map (has no units)]
        
        self.profit = 0 # [€]
        
        self.bidprice = 0 # Bid Price on the latest day [€/MW]
        self.bidvol = 0 # Bid Volume on the latest day [MW]
        
        self.health = 1 # 1-max health :::: 0.05-plant is decommissioned
        self.tik = 0 # Number of days plant has been active
        self.degrad_coeff = 0 # Ageing Coefficient of the power plant (will update later, initialized as 0)
        
        # print("Power plant {} created!".format(self.name))
        # when a source is created, increased by +1
        Source.population += 1
        Source.total_location.append(self.location)

In [ ]:
class Sink:
    
    """Represent a city"""
    population = 0
    total_location = []
    
    def __init__(self, name, demand, location=[]):
        
        self.name = name # Name of the sink (seller)
        self.demand = demand # Demand for the day [MW] 
        self.location = location # Location of the sink on the map (no units)
        
        # print("Power sink {} created!".format(self.name))
        # when a sink is created, increased by +1
        Sink.population += 1
        Sink.total_location.append(self.location)

# Function for calculating fuel prices based on day (**Nuclear < Coal < Gas**)

In [ ]:
def fuel_price(t): ### nuke,coal,gas,co2
    
    #random.seed(random.seed(t))
    
    ### raw data from history ###
    # Uranium
    raw_ux = np.array([0, 132.33, 339.31, 430.92, 468.24, 508.96, 587, 637.9, 1438.67, 1472.6, 1811.9, 1839.05, 2147.82, 2198.71, 2531.24, 2565.17, 3627.2, 3650.95, 4770.67, 5069.26, 5123.55, 5462.86, 5496.79, 5578.22, 5608.76, 5652.87, 5822.52, 5866.63, 5920.92, 6188.98, 6236.48, 6569])
    raw_uy = np.array([0.91, 0.96, 0.88, 0.88, 0.78, 0.86, 0.88, 0.83, 0.87, 0.74, 0.76, 0.71, 0.74, 0.69, 0.72, 0.64, 0.68, 0.62, 0.66, 0.64, 0.71, 0.72, 0.77, 0.79, 0.72, 0.8, 0.77, 0.87, 0.75, 0.8, 0.88, 0.91])
     
    # Coal from to
    raw_cx = np.array([-5.7, 20.11, 33.85, 75.02, 99.05, 126.47, 167.62, 193.4, 224.24, 258.58, 277.53, 320.4, 342.66, 402.78, 438.66, 455.82, 495.28, 522.81, 586.25, 605, 627.23, 654.68, 682.15, 706.12, 731.88, 759.29, 785.01, 809, 838.17, 860.39, 886.1, 913.53, 937.51, 961.55, 990.73, 1011.28, 1038.75, 1064.47, 1088.5, 1105.67]) 
    raw_cy = np.array([23.3, 22.8, 22.65, 22.58, 22.44, 22.51, 22.52, 22.23, 22.37, 22.09, 21.58, 21.58, 21.73, 21.15, 21.88, 21.81, 21.67, 21.16, 21.16, 21.82, 22.18, 22.11, 21.9, 22.12, 21.9, 22.05, 22.05, 22.12, 22.05, 22.42, 22.49, 22.57, 22.71, 22.5, 22.35, 22.5, 22.28, 22.28, 22.14, 22])

    # Gas from to
    raw_gx = np.array([-1.54, 38.82, 71.09, 102.5, 124.73, 167.61, 189.93, 229.35, 258.62, 291.43, 312.23, 344.88, 373.54, 401.3, 436.18, 465.14, 492.6, 555.32, 584.5, 606.68, 637.22, 654.26, 678.17, 684.86, 694.96, 703.37, 718.62, 730.5, 756.12, 773.33, 787.22, 799.37, 813.23, 832.26, 847.83, 863.4, 885.78, 911.56, 920.17, 937.3, 957.81, 979.95, 997.01, 1015.72, 1031.01, 1051.43, 1061.57, 1073.44, 1094.44])
    raw_gy = np.array([29.04, 33.69, 25.49, 22.29, 22.66, 22.59, 22.44, 22.52, 21.87, 20.49, 19.19, 18.83, 21.66, 19.77, 16.36, 17.45, 7.31, 21.53, 21.38, 22.04, 23.93, 24.58, 25.09, 26.11, 27.2, 28.14, 29.23, 29.96, 30.54, 30.18, 29.16, 28.29, 27.49, 26.48, 25.68, 24.88, 24.38, 24.02, 23.8, 23.95, 24.31, 25.18, 25.69, 26.56, 27.44, 28.31, 29.18, 29.98, 30.85])
    
    #CO2 price from 2009 to 2016 (0~2919)
    raw_co2x = np.array([-116.31, -40.64, 56.05, 152.75, 242.43, 323.71, 427.41, 514.29, 1086.04, 1136.49, 1425.17, 1514.85, 1612.95, 1699.83, 1792.32, 1981.5, 2064.18, 2167.88, 2253.36, 2334.64, 2427.13, 2523.82, 2617.71, 2703.19, 2794.28, 2889.57, 2982.06])
    raw_co2y = np.array([3.07, 3.36, 3.51, 3.2, 2.21, 2.06, 2.09, 1.87, 1.92, 1.95, 1.97, 2.78, 3.21, 2.7, 3.03, 5.01, 4.91, 5.28, 5.42, 5.52, 6.06, 7.48, 5.3, 4.56, 4.56, 3.56, 3.03])
    
    # evaluation of slope m and intercept b for linear regression line
    coeff_u = np.polyfit(raw_ux, raw_uy, 1)
    coeff_c = np.polyfit(raw_cx, raw_cy, 1)
    coeff_g = np.polyfit(raw_gx, raw_gy, 1)
    coeff_co2 = np.polyfit(raw_co2x, raw_co2y, 1)
    
    # Calculation of residual
    fit_u = np.polyval(coeff_u, raw_ux)
    rsd_u = np.sum((raw_uy-fit_u)**2)
    std_u = np.sqrt(rsd_u/len(raw_uy))

    fit_c = np.polyval(coeff_c, raw_cx)
    rsd_c = np.sum((raw_cy-fit_c)**2)
    std_c = np.sqrt(rsd_c/len(raw_cy))
    
    fit_g = np.polyval(coeff_g, raw_gx)
    rsd_g = np.sum((raw_gy-fit_g)**2)
    std_g = np.sqrt(rsd_g/len(raw_gy))
    
    fit_co2 = np.polyval(coeff_co2, raw_co2x)
    rsd_co2 = np.sum((raw_co2y-fit_co2)**2)
    std_co2 = np.sqrt(rsd_co2/len(raw_co2y))
    
    
    # interpolation
    time = np.linspace(0,364,num=365)
    fu = coeff_u[0]*time+coeff_u[1]+random.uniform(-std_u,std_u)
    fc = coeff_c[0]*time+coeff_c[1]+random.uniform(-std_c,std_c)
    fg = coeff_g[0]*time+coeff_g[1]+random.uniform(-std_g,std_g)
    fco2 = coeff_co2[0]*time+coeff_co2[1]+random.uniform(-std_co2,std_co2)
    
    # Unit conversion(Dollars or Cents to Euros & KWh to MW-day)
    u_fuel = ((0.01/1.2)/(0.001/24))*fu
    c_fuel = ((1/1.2)/(1/24))*fc
    g_fuel = ((1/1.2)/(1/24))*fg
    co2 = (1/1.2)*fco2
    
    fuel_list = []
    nuke_fuel = u_fuel[t]
    fuel_list.append(nuke_fuel)
    coal_fuel = c_fuel[t]
    fuel_list.append(coal_fuel)
    CCGT_fuel = g_fuel[t]
    fuel_list.append(CCGT_fuel)
    co2_cost = co2[t]
    fuel_list.append(co2_cost)
    
    return fuel_list

# Functions for generating Dummy Sources and Dummy Sinks (**AI**)

In [ ]:
####################################################################################################
###################################### Dummy source Generator ######################################
####################################################################################################

def dummy_source(n, size_map, fuel_list,pdata,AI_source_var):
    
    PPpw = pdata[1]
    PPcap = pdata[2]
    
    type_list = ["Nuclear", "Coal", "CCGT", "Hydro", "Wind", "Solar"]
    
    num_list = []
    num = random.randrange(1,n+1)
    i = 0
    for i in range(n):
        while num in num_list:
            num = random.randrange(1,n+1)
        num_list.append(num)
    
    
    dummy_list = [] #########################################################################  Dummy-source-name list generation
    i = 0
    for i in range(n):
        temp_name = random.choice(type_list)+str(num_list[i])+str("(AI)")
        temp_type = random.choice(type_list)
        dummy_list.append(temp_name)   
        ################################################################
    
    
    #########################  Dummy Source Generation  #####################################
    AI_sources = []
    nuke_fuel = fuel_list[0]
    coal_fuel = fuel_list[1]
    CCGT_fuel = fuel_list[2]
    CO2_cost = fuel_list[3]
    
    for x in range(n):
        #random.seed(x)
        if 'Nuclear' in dummy_list[x]:
            typ = 'Nuclear'
            pw = PPpw[0] * (1+(random.randrange(-AI_source_var,AI_source_var)/100))
            gen = nuke_fuel
            cap = PPcap[0]
            co2_emis = 0 # ton/MW per day
            co2_cost = 0
            xcoord = random.randrange(0,size_map)
            ycoord = random.randrange(0,size_map)
            AI_sources.append(Source(name=dummy_list[x], source=typ, power=pw, gen_cost=gen, cap_cost=cap, co2_emission=co2_emis, co2_cost=co2_cost, location=[xcoord,ycoord]))
            
        elif 'Coal' in dummy_list[x]:
            typ = 'Coal'
            pw = PPpw[1] * (1+(random.randrange(-AI_source_var,AI_source_var)/100))
            gen = coal_fuel
            cap = PPcap[1]
            co2_emis = PPco2_emis[1] # ton/MW per day
            co2_cost = CO2_cost
            xcoord = random.randrange(0,size_map)
            ycoord = random.randrange(0,size_map)
            AI_sources.append(Source(name=dummy_list[x], source=typ, power=pw, gen_cost=gen, cap_cost=cap, co2_emission=co2_emis, co2_cost=co2_cost, location=[xcoord,ycoord]))
            
            
        elif 'CCGT' in dummy_list[x]:
            typ = 'CCGT'
            pw = PPpw[2] * (1+(random.randrange(-AI_source_var,AI_source_var)/100))
            gen = CCGT_fuel
            cap = PPcap[2]
            co2_emis = PPco2_emis[2] # ton/MW per day
            co2_cost = CO2_cost
            xcoord = random.randrange(0,size_map)
            ycoord = random.randrange(0,size_map)
            AI_sources.append(Source(name=dummy_list[x], source=typ, power=pw, gen_cost=gen, cap_cost=cap, co2_emission=co2_emis, co2_cost=co2_cost, location=[xcoord,ycoord])) 
            
        elif 'Hydro' in dummy_list[x]:
            typ = 'Hydro'
            pw = PPpw[3] * (1+(random.randrange(-AI_source_var,AI_source_var)/100))
            gen = 0 
            cap = PPcap[3]
            co2_emis = 0 # ton/MW per day
            co2_cost = 0
            xcoord = random.randrange(0,size_map)
            ycoord = random.randrange(0,size_map)
            AI_sources.append(Source(name=dummy_list[x], source=typ, power=pw, gen_cost=gen, cap_cost=cap, co2_emission=co2_emis, co2_cost=co2_cost, location=[xcoord,ycoord]))

            
        elif 'Wind' in dummy_list[x]:
            typ = 'Wind'
            pw = PPpw[4] * (1+(random.randrange(-AI_source_var,AI_source_var)/100))
            gen = 0 
            cap = PPcap[4]
            co2_emis = 0 # ton/MW per day
            co2_cost = 0
            xcoord = random.randrange(0,size_map)
            ycoord = random.randrange(0,size_map)
            AI_sources.append(Source(name=dummy_list[x], source=typ, power=pw, gen_cost=gen, cap_cost=cap, co2_emission=co2_emis, co2_cost=co2_cost, location=[xcoord,ycoord])) 
            
        else: 
            typ = 'Solar'
            pw = PPpw[5] * (1+(random.randrange(-AI_source_var,AI_source_var)/100))
            gen = 0 
            cap = PPcap[5]
            co2_emis = 0 # ton/MW per day
            co2_cost = 0
            xcoord = random.randrange(0,size_map)
            ycoord = random.randrange(0,size_map)
            AI_sources.append(Source(name=dummy_list[x], source=typ, power=pw, gen_cost=gen, cap_cost=cap, co2_emission=co2_emis, co2_cost=co2_cost, location=[xcoord,ycoord]))
            
    return dummy_list, AI_sources

In [ ]:
####################################################################################################
###################################### Dummy sink Generator ########################################
####################################################################################################

def dummy_sink(n,size_map,AI_sink_var):
    
    type_list = ["Metropolis", "city", "town", "village"]
    adj_list = ["Happy_", "Unhappy_", "Crowded_", "Boring_", "Quarantined_"]
    
    num_list = []
    num = random.randrange(1,n+1)
    i = 0
    for i in range(n):
        while num in num_list:
            num = random.randrange(1,n+1)
        num_list.append(num)
    
    dummy_list = [] #########################################################################  Dummy-sink-name list generation
    for i in range(n):
        temp_name = random.choice(adj_list)+random.choice(type_list)+str(num_list[i])+str("(AI)")
        temp_type = random.choice(type_list)
        dummy_list.append(temp_name)   
        ################################################################
    
    
    #########################  Dummy Sink Generation  #####################################
    AI_sinks = []
    for x in range(n):
        #random.seed(x)
        if 'Metropolis' in dummy_list[x]:
            temp_demand = 400 * (1+(random.randrange(-AI_sink_var,AI_sink_var)/100))
            xcoord = random.randrange(0,size_map)
            ycoord = random.randrange(0,size_map)
            AI_sinks.append(Sink(name=dummy_list[x], demand = temp_demand, location=[xcoord,ycoord]))
            
        
        elif 'city' in dummy_list[x]:
            temp_demand = 250 * (1+(random.randrange(-AI_sink_var,AI_sink_var)/100))
            xcoord = random.randrange(0,size_map)
            ycoord = random.randrange(0,size_map)
            AI_sinks.append(Sink(name=dummy_list[x], demand = temp_demand, location=[xcoord,ycoord]))
            
            
        elif 'town' in dummy_list[x]:
            temp_demand = 150 * (1+(random.randrange(-AI_sink_var,AI_sink_var)/100))
            xcoord = random.randrange(0,size_map)
            ycoord = random.randrange(0,size_map)
            AI_sinks.append(Sink(name=dummy_list[x], demand = temp_demand, location=[xcoord,ycoord]))
        
        else:
            temp_demand = 100 * (1+(random.randrange(-AI_sink_var,AI_sink_var)/100))
            xcoord = random.randrange(0,size_map)
            ycoord = random.randrange(0,size_map)
            AI_sinks.append(Sink(name=dummy_list[x], demand = temp_demand, location=[xcoord,ycoord]))
        
    
    return dummy_list, AI_sinks

# Functions for generating AI selling and buying Bids

In [ ]:
def sell_bids(AI_sources,dummy_source_list,AI_sell_bid_prof):
    n = len(dummy_source_list)
    dm_seller_bids = []
    for n in range(n):
        temp_vol = math.trunc(AI_sources[n].power)
        
        if AI_sources[n].source_type == 'Hydro' or AI_sources[n].source_type == 'Wind' or AI_sources[n].source_type == 'Solar':
            temp_price = AI_sources[n].power*(1+random.randrange(AI_sell_bid_prof[0],AI_sell_bid_prof[1])/100)
        else:
            temp_price = (AI_sources[n].gen_cost + AI_sources[n].co2_emission*AI_sources[n].co2_cost)*(1+random.randrange(AI_sell_bid_prof[0],AI_sell_bid_prof[1])/100)
            
        dm_seller_bids.append([temp_price,temp_vol])
    
    return dm_seller_bids

In [ ]:
def buy_bids(AI_sinks,dummy_sink_list,AI_buy_bid_impor):
    n = len(dummy_sink_list)
    dm_buyer_bids = []
    for n in range(n):
        temp_vol = math.trunc(AI_sinks[n].demand)
        temp_price = (1+(random.randrange(0,AI_buy_bid_impor)/100)*(2.5e4/math.sqrt(AI_sinks[n].demand)))
        dm_buyer_bids.append([temp_price,temp_vol])
    
    return dm_buyer_bids

# Merit Order calculation

In [ ]:
def merit_order(sell_bids, buy_bids):     ### input: two bid-lists of seller and buyer in random order -> output: tow merit-ordered bid-lists
    temp1 = sell_bids
    for i in range(len(temp1)):
        temp1[i][1] = -temp1[i][1]
    temp1 = sorted(temp1)
    for i in range(len(temp1)):
        temp1[i][1] = -temp1[i][1]
        
    temp2 = sorted(buy_bids, reverse=True)
    
    return temp1, temp2

# Cumulative Volume Calculation for Bids

In [ ]:
def cumulative_volume(sort_sbids, sort_bbids):
    snum = len(sort_sbids)
    bnum = len(sort_bbids)
    
    ###  Generation of bid lists with cumulative volume ###
    temp_sbids = sort_sbids
    temp_bbids = sort_bbids
    for g in range(snum-1):
        temp_sbids[g+1][1] = temp_sbids[g+1][1] + temp_sbids[g][1]
    for h in range(bnum-1):
        temp_bbids[h+1][1] = temp_bbids[h+1][1] + temp_bbids[h][1]
        
        
    ### switching the places of price and volume ###
    temp = 0
    for i in range(snum):
        temp_sbids[i][0], temp_sbids[i][1] = temp_sbids[i][1], temp_sbids[i][0]
    
    temp = 0
    for j in range(bnum):
        temp_bbids[j][0], temp_bbids[j][1] = temp_bbids[j][1], temp_bbids[j][0]

    return  temp_sbids, temp_bbids

# Function to Calculate MCV and MCP

In [ ]:
def market(sbid,bbid,day,choice2):        ###Step.1: Find MCV and MCP
    
    snum = len(sbid)
    bnum = len(bbid)

    sbid = np.asarray(sbid)
    bbid = np.asarray(bbid)
    
    
    ##### Scaling down #####
    sbid_max = np.amax(sbid,axis=0)
    bbid_max = np.amax(bbid,axis=0)
    bid_max = np.amax([sbid_max,bbid_max],axis=0)
    
    sbid = sbid/bid_max
    bbid = bbid/bid_max
    ########################
    
    
    s_idx = int(snum/2) ### Initial guess - seller index (midpoint)
    b_idx = int(bnum/2) ### Initial guess - buyer  index (midpoint)
    
    MCV_1 = (sbid[s_idx][0] + bbid[b_idx][0])/2
    MCP_1 = (sbid[s_idx][1] + bbid[b_idx][1])/2
    
    itr = 1
    
    ######################### Recursive Algorithim #############################

    h = 100 ### Refinement level of Recursive Algorithm
    
    l_range = int((snum+bnum)/10)
    r_range = int((snum+bnum)/10)
    
    l_range_fine = int(math.sqrt(l_range))
    r_range_fine = int(math.sqrt(r_range))

    tol = 1e10 #Initial Tolerance
    tol_val = 1e-5 # Final Tolerance limit for Convergence

    #############################################################################

    MCVtemp = [MCV_1]
    MCPtemp = [MCP_1]
    
    ###################### Coarse MCP-MCV Calclation Part #######################
    
    while tol>tol_val:
        
        ### Condtion for making sure index range of l_range and r_range does not exceed size of Seller and Buyer Bids ###
        while (s_idx-l_range) < 0:
            l_range = int(l_range*0.75)
            
        while (s_idx+r_range) > snum:
            r_range = int(r_range*0.75)
            
        while (b_idx-l_range) < 0:
            l_range = int(l_range*0.75)
            
        while (b_idx+r_range) > bnum:
            r_range = int(r_range*0.75)
        ##################################################################################################################
        
        sbidtemp = [sbid[s_idx+i][:] for i in range(-l_range,r_range)]
        bbidtemp = [bbid[b_idx+i][:] for i in range(-l_range,r_range)]
        
        M = np.zeros((len(sbidtemp),len(bbidtemp)))
        
        for i in range(len(sbidtemp)):
            for j in range(len(bbidtemp)):
                
                M[i,j] = pow((sbidtemp[i][0] - bbidtemp[j][0]),2) + pow((sbidtemp[i][1] - bbidtemp[j][1]),2)
        
        M_min = np.amin(M) ### arr2D == numpy.amin(arr2D)
        idx = np.argwhere(M==M_min) ### find the indices of sell_bid and buy_bid to determine the MCP and MCV
        
        if len(idx) > 1:
            s_idx = int(idx[-1,0])
            b_idx = int(idx[-1,1])

        elif len(idx) == 1:
            s_idx = int(idx[0,0])
            b_idx = int(idx[0,1])
            
        MCVtemp.append((sbidtemp[s_idx][0] + bbidtemp[b_idx][0])/2)
        MCPtemp.append((sbidtemp[s_idx][1] + bbidtemp[b_idx][1])/2)

        tol = pow((MCVtemp[-1]-MCVtemp[-2]),2) + pow((MCPtemp[-1]-MCPtemp[-2]),2)
        
        diff_s = np.abs(sbid - sbidtemp[s_idx])
        min_s =  np.amin(diff_s)
        idx_s = np.argwhere(diff_s==min_s)
        s_idx = idx_s.flatten()[0]
        
        diff_b = np.abs(bbid - bbidtemp[b_idx])
        min_b =  np.amin(diff_b)
        idx_b = np.argwhere(diff_b==min_b)
        b_idx = idx_b.flatten()[0]
        
        itr = itr+1
    ###################### Fine MCP-MCV Calclation Part #######################
        
    sbid_vic=[]
    bbid_vic=[]

    sbid_vic = np.asarray([[0,0]])
    bbid_vic = np.asarray([[0,0]])
        
    for ii in range(-l_range_fine,r_range_fine):

        s1_vic = np.stack((((np.linspace(sbid[s_idx-1+ii][0],sbid[s_idx+ii][0],h)),np.full((h),sbid[s_idx+ii][1]))),axis=1)
        s2_vic = np.stack(((np.full((h),sbid[s_idx+ii][0]),(np.linspace(sbid[s_idx+ii][1],sbid[s_idx+1+ii][1],h)))),axis=1)

        sbid_vic = np.concatenate((sbid_vic,s1_vic,s2_vic),axis=0)

        b1_vic = np.stack((((np.linspace(bbid[b_idx-1+ii][0],bbid[b_idx+ii][0],h)),np.full((h),bbid[b_idx+ii][1]))),axis=1)
        b2_vic = np.stack(((np.full((h),bbid[b_idx+ii][0]),(np.linspace(bbid[b_idx+ii][1],bbid[b_idx+1+ii][1],h)))),axis=1)

        bbid_vic = np.concatenate((bbid_vic,b1_vic,b2_vic),axis=0)

    sbid_vic = np.delete(sbid_vic,0,0)
    bbid_vic = np.delete(bbid_vic,0,0)

    snum_vic = len(sbid_vic)
    bnum_vic = len(bbid_vic)

    M_vic = np.zeros((snum_vic,bnum_vic))

    for i_vic in range(snum_vic):
        for j_vic in range(bnum_vic):

            M_vic[i_vic,j_vic] = pow((sbid_vic[i_vic,0] - bbid_vic[j_vic,0]),2) + pow((sbid_vic[i_vic,1] - bbid_vic[j_vic,1]),2)

    M_min_vic = np.amin(M_vic) ### arr2D == numpy.amin(arr2D)
    idx_vic = np.argwhere(M_vic==M_min_vic) ### find the indices of sell_bid and buy_bid to determine the MCP and MCV

    if len(idx_vic) > 1:
        s_idx_vic = int(idx_vic[-1,0])
        b_idx_vic = int(idx_vic[-1,1])

    elif len(idx_vic) == 1:
        s_idx_vic = int(idx_vic[0,0])
        b_idx_vic = int(idx_vic[0,1])


    MCVtemp.append((sbid_vic[s_idx_vic][0] + bbid_vic[b_idx_vic][0])/2)
    MCPtemp.append((sbid_vic[s_idx_vic][1] + bbid_vic[b_idx_vic][1])/2)
        
    ####################################################################
    
    ##### Re-Scaling back up #####
    sbid = sbid*bid_max
    bbid = bbid*bid_max
    sbid_vic = sbid_vic*bid_max                   
    bbid_vic = bbid_vic*bid_max
    MCVtemp = np.asarray(MCVtemp)*bid_max[0]
    MCPtemp = np.asarray(MCPtemp)*bid_max[1]
    ##############################
    
    
    MCV = MCVtemp[-1]
    MCP = MCPtemp[-1]
    
    plt.figure()
    
    s1,s2 = sbid[:,0],sbid[:,1]
    b1,b2 = bbid[:,0],bbid[:,1]
    
    s1_temp,s2_temp = sbid_vic[:,0],sbid_vic[:,1]
    b1_temp,b2_temp = bbid_vic[:,0],bbid_vic[:,1]
    
    plt.step(s1,s2,color='blue',label="Seller Bids")
    plt.step(b1,b2,color='black',label="Buyer Bids")
    #plt.plot(s1,s2,'go') ### enable this line to PLOT ALL SELLER POINTS
    #plt.plot(b1,b2,'go') ### enable this line to PLOT ALL BUYER POINTS
    #plt.step(s1_temp,s2_temp,'y') ### enable this line to HIGHLIGHT INTERSECTION VICINITY (Seller) 
    #plt.step(b1_temp,b2_temp,'y') ### enable this line to HIGHLIGHT INTERSECTION VICINITY (Buyer)
    #plt.plot(MCVtemp,MCPtemp,'yo') ### enable this line to SEE HOW THE ITERATIVE ALGORITHM WORKS
    info = str("MCV: " + str('{:,.2f}'.format(MCV)) +" MW" + '\n' + "MCP: " + str('{:,.2f}'.format(MCP)) + " €")
    plt.plot(MCV,MCP,'ro',label=info)
    plt.xlabel('Volume(MW)')
    plt.ylabel('Price(€/MW)')
    plt.legend(loc="upper left")
    
    if choice2 == 1:
        plt.savefig('demand supply graph Day {}.png'.format(day)) # Export the demand and supply curve
    
    plt.show()
    
    return MCV,MCP

# Function for updating Fuel Prices

In [ ]:
def update_fuel(pp,fuel): # (powerplant[list of all powerplants] , fuel)
    
    
    for p in pp:
        
        p.co2_cost = fuel[3]
        
        if p.source_type == 'Nuclear':
            p.gen_cost = fuel[0]
        
        elif p.source_type == 'Coal':
            p.gen_cost = fuel[1]
            
        elif p.source_type == 'CCGT':
            p.gen_cost = fuel[2]
            

# Function for updating Intermittent Sources capacity for the day

In [ ]:
def update_renewable(pp,hydro_prob,wind_prob,solar_prob):
    
    for p in pp:
        
        if p.source_type == 'Hydro':
            p.power_day = p.power * (1+(random.randrange(-hydro_prob,0)/100)) # up to -30 %
        
        if p.source_type == 'Wind':
            p.power_day = p.power * (1+(random.randrange(-wind_prob,0)/100)) # up to -50 %
            
        if p.source_type == 'Solar':
            p.power_day = p.power * (1+(random.randrange(-solar_prob,0)/100))  # up to -70 %
            

# Function for updating Weather Data - Alternative for Intermittent Source 

In [ ]:
def weather_data(wsize,pp):
    
    div = 50
    
    plt.figure(figsize=(20,5))
    
    X,Y = np.meshgrid(np.linspace(0,wsize,div),np.linspace(0,wsize,div))
    x_rand = random.randrange(-wsize,0)
    y_rand = random.randrange(-wsize,0)
    
    solar = np.array([math.exp(-(pow(((x+x_rand)/wsize),2)+(pow(((y+y_rand)/wsize),2)))) for x,y in zip(np.ravel(X),np.ravel(Y))])
    solar = solar.reshape(X.shape)
    
    wind = np.array([math.exp(-(pow(((x+x_rand)/(x_rand-0.01)),2)+(pow(((y+y_rand)/(y_rand-0.01)),2)))) for x,y in zip(np.ravel(X),np.ravel(Y))])
    wind = wind.reshape(X.shape)
    
    rain = np.array([(1-math.exp(-(pow(((x+x_rand)/wsize),2)+(pow(((y+y_rand)/wsize),2))))) for x,y in zip(np.ravel(X),np.ravel(Y))])
    rain = rain.reshape(X.shape)
    
    
    for p in pp:
        
        if p.source_type == 'Solar':
            
            plt.subplot(131)
            x_idx = np.argwhere(abs(X[0,:] - p.location[0]) == np.amin(abs(X[0,:] - p.location[0])))
            y_idx = np.argwhere(abs(Y[:,0] - p.location[1]) == np.amin(abs(Y[:,0] - p.location[1])))
            x_idx = int(x_idx[-1])
            y_idx = int(y_idx[-1])

            p.power_day = p.power * solar[x_idx,y_idx]
        
            plt.plot(X[0,x_idx],Y[y_idx,0],'ro')
            plt.annotate(p.name,xy=(X[0,x_idx],Y[y_idx,0]))
            plt.pcolor(X,Y,solar)
            plt.colorbar()
            plt.title('Solar Power Density')
            
        elif p.source_type == 'Wind':
            
            plt.subplot(132)
            x_idx = np.argwhere(abs(X[0,:] - p.location[0]) == np.amin(abs(X[0,:] - p.location[0])))
            y_idx = np.argwhere(abs(Y[:,0] - p.location[1]) == np.amin(abs(Y[:,0] - p.location[1])))
            x_idx = int(x_idx[-1])
            y_idx = int(y_idx[-1])

            p.power_day = p.power * wind[x_idx,y_idx]

            plt.plot(X[0,x_idx],Y[y_idx,0],'ro')
            plt.annotate(p.name,xy=(X[0,x_idx],Y[y_idx,0]))
            plt.pcolor(X,Y,wind)
            plt.colorbar()
            plt.title('Wind Power Density')
            
        elif p.source_type == 'Hydro':
            
            plt.subplot(133)
            x_idx = np.argwhere(abs(X[0,:] - p.location[0]) == np.amin(abs(X[0,:] - p.location[0])))
            y_idx = np.argwhere(abs(Y[:,0] - p.location[1]) == np.amin(abs(Y[:,0] - p.location[1])))
            x_idx = int(x_idx[-1])
            y_idx = int(y_idx[-1])

            p.power_day = p.power * rain[x_idx,y_idx]

            plt.plot(X[0,x_idx],Y[y_idx,0],'ro')
            plt.annotate(p.name,xy=(X[0,x_idx],Y[y_idx,0]))
            plt.pcolor(X,Y,rain)
            plt.colorbar()
            plt.title('Rain Data')
        
    plt.show()

# Function for asking Player Bids

In [ ]:
def ask_bids(pp,day,fuel):
    
    pbids = []
    
    for p in pp:
        
    
        print(("\n" + p.name),end='\n')
        print(("Plant health : " + str('{:,.2f}'.format(p.health*100)) + " %"),end ='\n')
        
        if p.source_type == 'Nuclear':
            
            print(("Nuclear Fuel Price = "+str('{:,.2f}'.format(fuel[0]))+" €"), end='\n')
            
            print(("Max possible Generation for today is: " + str('{:,.2f}'.format(p.power))+" MW"),end='\n')
            p.bidvol = float(input("Please enter BID VOLUME (MW): "))
            
            print(("Generation cost is: " + str('{:,.2f}'.format(p.gen_cost))+" €"), end='\n')
            p.bidprice = float(input("Please enter the BID PRICE per MW (€): "))
            
            pbids.append([p.bidprice,p.bidvol])
            
        if p.source_type == 'Coal':
            
            print(("Coal Fuel Price = "+str('{:,.2f}'.format(fuel[1]))+" €"), end='\n')
            
            print(("Max possible Generation for today is: " + str('{:,.2f}'.format(p.power))+" MW"),end='\n')
            p.bidvol = float(input("Please enter BID VOLUME (MW): "))
            
            print(("Generation cost with CO2 tax for bidding volume is: " + str('{:,.2f}'.format(p.gen_cost+(p.co2_emission*p.co2_cost)))+" €"), end='\n')
            p.bidprice = float(input("Please enter the BID PRICE per MW (€): "))
            
            pbids.append([p.bidprice,p.bidvol])
            
        if p.source_type == 'CCGT':
            
            print(("CCGT Fuel Price = "+str('{:,.2f}'.format(fuel[2]))+" €"), end='\n')
            
            print(("Max possible Generation for today is: " + str('{:,.2f}'.format(p.power))+" MW"),end='\n')
            p.bidvol = float(input("Please enter BID VOLUME (MW): "))
            
            print(("Generation cost with CO2 tax for bidding volume is: " + str('{:,.2f}'.format(p.gen_cost+(p.co2_emission*p.co2_cost)))+" €"), end='\n')
            p.bidprice = float(input("Please enter the BID PRICE per MW (€): "))
            
            pbids.append([p.bidprice,p.bidvol])
            
        if p.source_type == 'Hydro':
            
            print(("Max possible Generation for today is: " + str('{:,.2f}'.format(p.power_day))+" MW"),end='\n')
            p.bidvol = float(input("Please enter BID VOLUME (MW): "))
            
            p.bidprice = float(input("Please enter the BID PRICE per MW (€): "))
            pbids.append([p.bidprice,p.bidvol])
            
        if p.source_type == 'Wind':
            
            print(("Max possible Generation for today is: " + str('{:,.2f}'.format(p.power_day))+" MW"),end='\n')
            p.bidvol = float(input("Please enter BID VOLUME (MW): "))
            
            p.bidprice = float(input("Please enter the BID PRICE per MW (€): "))
            pbids.append([p.bidprice,p.bidvol])
            
        if p.source_type == 'Solar':
            
            print(("Max possible Generation for today is: " + str('{:,.2f}'.format(p.power_day))+" MW"),end='\n')
            p.bidvol = float(input("Please enter BID VOLUME (MW): "))
            
            p.bidprice = float(input("Please enter the BID PRICE per MW (€): "))
            pbids.append([p.bidprice,p.bidvol])
            
        if p.bidprice == 0 or p.bidvol == 0: ### This statement must always be in last
            pbids.pop(-1)
    
    return pbids

# Function for aging power plant

In [ ]:
def age_pp(pp,pp_decom,MCP,pdata):
    
    PPdeg_coeff = pdata[4]

    for p in pp:
        
        p.tik = p.tik + 1
        
        ##### Specifying Various Degradation Coefficients #####
        if p.source_type == 'Nuclear':
            p.degrad_coeff = PPdeg_coeff[0]
        
        elif p.source_type == 'Coal':
            p.degrad_coeff = PPdeg_coeff[1]
            
        elif p.source_type == 'CCGT':
            p.degrad_coeff = PPdeg_coeff[2]
            
        elif p.source_type == 'Hydro':
            p.degrad_coeff = PPdeg_coeff[3]
            
        elif p.source_type == 'Wind':
            p.degrad_coeff = PPdeg_coeff[4]
            
        else: ### Solar ###
            p.degrad_coeff = PPdeg_coeff[5]
        #########################################################
        
        
        if p.health < 0.05:
            
            print("\n" + "The power plant " + p.name + " has reached its maximum lifespan and will be destroyed")
            
            (pp.decom).extend(p)
            pp.remove(p)
            
        else:
            
            if (p.bidvol > 0) and (p.bidprice <= MCP):

                percent = p.bidvol/p.power # operating capacity
                p.health = (1 - percent * p.degrad_coeff) * p.health # % of remaining plant health is sacrificed everytime plant is operated at max capacity based on parameter degrading coeff

                if p.health >= 0.5:

                    p.cur_val = p.health * p.cur_val # capital cost decreases according to plant health
                    p.power = p.health * p.power # max generating capacity decreases according to plant health
                    p.co2_emission = (1 - p.health + 1) * p.co2_emission  # co2 emmission level worsens according to plant health
                    
    return pp, pp_decom            

# Function for checking if bids have been cleared

In [ ]:
def check_bids(pp,MCV,MCP):
    
    for p in pp:
        

        if p.bidprice == 0 or p.bidvol == 0:
            continue
        
        else:
            
            if p.bidprice <= MCP:

                print("\n" + "Your bid for " + p.name + " has been ACCEPTED")
                p.profit = p.profit + (MCP - p.gen_cost - p.co2_emission*p.co2_cost)*p.bidvol
                print("Your total profit for the plant is: " + str('{:,.2f}'.format(p.profit))+" €")

            else:

                print("\n" + "Your bid for " + p.name + " has been REJECTED")

# Function for buying new powerplants

In [ ]:
def buy_plant(pp,pfund,pdata):
    
    inp = str(input("\n" + "Do you want to BUY a power plant? (yes/no) : "))
    
    while inp.lower() == "yes":
        
        PPtyp = pdata[0]
        PPpw = pdata[1]
        PPcap = pdata[2]
        PPco2_emis = pdata[3]
        PPdeg_coeff = pdata[4]
        PPxcoord = pdata[5]
        PPycoord = pdata[6]
        
        
        data = [[PPtyp[i],PPpw[i],str('{:,.2f}'.format(PPcap[i])),PPco2_emis[i],(100*PPdeg_coeff[i])] for i in range(6)]
        print(tabulate(data,headers=["Type","Production Capacity(MW)","Power Plant Cost(€)","CO2 emission(ton/MW)","Ageing Percentage(%)"],tablefmt="pretty"))
        print("\n"+"Remaining funds : "+str('{:,.2f}'.format(pfund))+" M€"+"\n")

        w_all = [int(x) for x in input("Input the number of plants to purchase in prescribed order - [Nuclear,Coal,CCGT,Hydro,Wind,Solar] : ").split()]

        cap = [cap for cap in PPcap]

        total = [((int(w_all[i]))* float(cap[i])) for i in range(len(cap))]



        if sum(total) <= pfund:
            pfund = pfund - sum(total)
            print("\n"+"Remaining Funds : "+str('{:,.2f}'.format(pfund))+" €"+"\n")

        else:
            print("\n"+"You lack the funds to make this purchase!!!" + "\n")
            continue
        
        inp = str(input("Do you want to BUY a power plant? (yes/no) : "))
        
        pp_dummy = []
    
        ii = 0
        for w in w_all:
            for q in range(w):
                s = 0
                for p in pp:
                    if p.source_type == PPtyp[ii]:
                        s += 1
                temp_name = 'PP' + str(PPtyp[ii]) + str(s+q+1)
                pp_dummy.append(Source(name=temp_name,source=PPtyp[ii],power=PPpw[ii],gen_cost=0,cap_cost=PPcap[ii],co2_emission=PPco2_emis[ii],co2_cost=0,location=[PPxcoord[ii],PPycoord[ii]]))
            ii+=1
        pp.extend(pp_dummy)

    return pp,pfund

# Function for selling power plants

In [ ]:
def sell_plant(pp,pfund):
    
    while len(pp)!=0:
    
        inp = str(input("\n" + "Do you want to SELL a power plant? (yes/no) : "))

        if inp.lower() == "yes":

            print("\n" + "List of your power plants owned:" + "\n")

            p_idx = 0

            for p in pp:

                print(str(p.name) + " - Press " + str(p_idx) + " to Sell")
                p_idx = p_idx + 1

            inp_1 = int(input("\n" + "Which plant do you want to SELL : "))
            temp_p = pp.pop(inp_1)

            print("Power Plant " + str(temp_p.name) + " has been sold")
            pfund = pfund + temp_p.profit
            pfund = pfund + temp_p.cur_val
            print("\n"+"Remaining Funds : "+str('{:,.2f}'.format(pfund))+" €"+"\n")
            
        else:
            break
    
    return pp,pfund

# Function for viewing portfolio

In [ ]:
def view_portfolio(pp,pp_decom,fund):
    
    inp = str(input("\n" + "Do you want to view your portfolio (yes/no) : "))
        
    if inp.lower() == "yes":
            
        print("\n"+"Remaining funds : "+str('{:,.2f}'.format(fund))+" €"+"\n")
        print("\n"+" ***** List of Active Power Plants ***** "+"\n")
        data = [[p.name,p.source_type,str('{:,.2f}'.format(p.power)),str('{:,.2f}'.format(p.cur_val)),str('{:,.2f}'.format(p.co2_emission)),(100*p.degrad_coeff),str('{:,.2f}'.format(100*p.health)),str('{:,.2f}'.format(p.profit))] for p in pp]
        print(tabulate(data,headers=["Name","Type","Capacity(MW)","Value(€)","CO2 emission(ton/MW)","Ageing Percent(%)","Health(%)","Profit(€)"],tablefmt="pretty"))
        print("\n")
        
        if len(pp_decom) != 0:
            
            print("\n"+" ***** List of Decommissioned Power Plants ***** "+"\n")
            data = [[p.name,p.source_type,str('{:,.2f}'.format(p.power)),str('{:,.2f}'.format(p.cur_val)),str('{:,.2f}'.format(p.co2_emission)),(100*p.degrad_coeff),str('{:,.2f}'.format(100*p.health)),str('{:,.2f}'.format(p.profit))] for p in pp_decom]
            print(tabulate(data,headers=["Name","Type","Capacity(MW)","Value(€)","CO2 emission(ton/MW)","Ageing Percent(%)","Health(%)","Profit(€)"],tablefmt="pretty"))
            print("\n")

# Function for exporting Data

In [ ]:
### Export of Source-information ####
def export_info(pname, pfund, t, AI_sorc, AI_sink, MCV, MCP, pp, ppd, His_MCP, His_MCV, His_Fuel, exp_sellbids, exp_buybids):
    data = xlsxwriter.Workbook("market_data Day_{}.xls".format(t))
    
    ### AI sources export ###
    sheet1 = data.add_worksheet("AI source info Day {}".format(t))
    
    sheet1.write(0, 0, 'Name')
    sheet1.write(0, 1, 'Type')
    sheet1.write(0, 2, 'Power[MW]')
    sheet1.write(0, 3, 'Gen_cost[€/MW]')
    sheet1.write(0, 4, 'Cap_cost[€]')
    sheet1.write(0, 5, 'CO2_emission[ton/MW]')
    sheet1.write(0, 6, 'CO2_cost[€/ton]')
    sheet1.write(0, 7, 'Location(x-coord)')
    sheet1.write(0, 8, 'Location(y-coord)')
    
    for i in range(len(AI_sorc)):
        sheet1.write(i+1, 0, AI_sorc[i].name)
        sheet1.write(i+1, 1, AI_sorc[i].source_type)
        sheet1.write(i+1, 2, AI_sorc[i].power)
        sheet1.write(i+1, 3, AI_sorc[i].gen_cost)
        sheet1.write(i+1, 4, AI_sorc[i].cap_cost)
        sheet1.write(i+1, 5, AI_sorc[i].co2_emission)
        sheet1.write(i+1, 6, AI_sorc[i].co2_cost)
        loc = AI_sorc[i].location
        xcood = loc[0]
        ycoord = loc[1]
        sheet1.write(i+1, 7, loc[0])
        sheet1.write(i+1, 8, loc[1])
    
    
    
    
    
    ### AI sinks export ###
    sheet2 = data.add_worksheet("AI sink info Day {}".format(t))
    
    sheet2.write(0, 0, 'Name')
    sheet2.write(0, 1, 'Demand[MW]')
    sheet2.write(0, 2, 'Location(x-coord)')
    sheet2.write(0, 3, 'Location(y-coord)')
    
    for j in range(len(AI_sink)):
        sheet2.write(j+1, 0, AI_sink[j].name)
        sheet2.write(j+1, 1, AI_sink[j].demand)
        loc = AI_sink[j].location
        xcood = loc[0]
        ycoord = loc[1]
        sheet2.write(j+1, 2, loc[0])
        sheet2.write(j+1, 3, loc[1])
    
    
    
    
    ### MCP and MCV export ###
    sheet3 = data.add_worksheet("MCV and MCP Day {}".format(t))
    
    sheet3.write(0, 0, 'Day')
    sheet3.write(0, 1, 'MCP[€/MW]')
    sheet3.write(0, 2, 'MCV[MW]')
    sheet3.write(1, 0, t)
    sheet3.write(1, 1, MCP)
    sheet3.write(1, 2, MCV)
    
    sheet3.write(0, 4, 'Seller bids')
    sheet3.write(0, 5, 'Volume[MW]')
    sheet3.write(0, 6, 'Price[€/MW]')
    #sheet3.write(0, 7, 'Cumulative Volume')
    
    sheet3.write(0, 10, 'Buyer bids')
    sheet3.write(0, 11, 'Volume[MW]')
    sheet3.write(0, 12, 'Price[€/MW]')
    
    #write ref. data in a table
    for jjj in range(len(exp_sellbids)):
        sheet3.write(jjj+1,5,exp_sellbids[jjj][0])
        sheet3.write(jjj+1,6,exp_sellbids[jjj][1])
        
    for kk in range(len(exp_buybids)):
        sheet3.write(kk+1,11,exp_buybids[kk][0])
        sheet3.write(kk+1,12,exp_buybids[kk][1])

     # plotting from the table(charts are independent of worksheets)  
    chart0 = data.add_chart({'type': 'line'})
    chart0.set_y_axis({'name': 'Price[€/MW]'})
    chart0.set_x_axis({'name': 'Volume[MW]'})
    chart0.set_title({'name': 'supply and demand curve'})
    
    data_start_loc11 = [1, 6] # xlsxwriter requires list, no tuple
    data_end_loc11 = [data_start_loc11[0] + len(exp_sellbids), 6]
    data_start_loc12 = [1, 5] # xlsxwriter requires list, no tuple
    data_end_loc12 = [data_start_loc12[0] + len(exp_sellbids), 5]
    chart0.add_series({
        'values': [sheet3.name] + data_start_loc11 + data_end_loc11,
        'categories': [sheet3.name] + data_start_loc12 + data_end_loc12,
        'name': "supply bids",  
    })
    
    data_start_loc21 = [1, 12] # xlsxwriter requires list, no tuple
    data_end_loc21 = [data_start_loc21[0] + len(exp_buybids), 12]
    data_start_loc22 = [1, 11] # xlsxwriter requires list, no tuple
    data_end_loc22 = [data_start_loc22[0] + len(exp_buybids), 11]
    chart0.add_series({
        'values': [sheet3.name] + data_start_loc21 + data_end_loc21,
        'categories': [sheet3.name] + data_start_loc22 + data_end_loc22,
        'name': "demand bids",  
    })
    sheet3.insert_chart('O1', chart0)   
    
        
        
        
        
        
    
    
    ### player sources export ###
    sheet4 = data.add_worksheet("Player source info Day {}".format(t))
    
    
    sheet4.write(0, 0, 'Player Name')
    sheet4.write(0, 1, 'Player Fund[€]')
    sheet4.write(1, 0, pname)
    sheet4.write(1, 1, pfund)
    
    
    sheet4.write(3, 1, 'Name')
    sheet4.write(3, 2, 'Type')
    sheet4.write(3, 3, 'Power[MW]')
    sheet4.write(3, 4, 'Daily Max Power[MW]')    
    sheet4.write(3, 5, 'Health')
    sheet4.write(3, 6, 'Gen_cost[€/MW]')
    sheet4.write(3, 7, 'Cap_cost[€]')
    sheet4.write(3, 8, 'Curent Value[€]')
    sheet4.write(3, 9, 'CO2_emission[ton/MW]')
    sheet4.write(3, 10, 'CO2_cost[€/ton]')
    sheet4.write(3, 11, 'Price bid[€/MW]')
    sheet4.write(3, 12, 'Volume bid[MW]')
    sheet4.write(3, 13, 'Profit[€]')
    sheet4.write(3, 14, 'Location(x-coord)')
    sheet4.write(3, 15, 'Location(y-coord)')
    sheet4.write(3, 16, 'Active Days')
    
    sheet4.write(4, 0, 'Active Power Plants')
    for k in range(len(pp)):
        sheet4.write(k+4, 1, pp[k].name)
        sheet4.write(k+4, 2, pp[k].source_type)
        sheet4.write(k+4, 3, pp[k].power)
        sheet4.write(k+4, 4, pp[k].power_day) 
        sheet4.write(k+4, 5, pp[k].health)
        sheet4.write(k+4, 6, pp[k].gen_cost)
        sheet4.write(k+4, 7, pp[k].cap_cost)
        sheet4.write(k+4, 8, pp[k].cur_val)        
        sheet4.write(k+4, 9, pp[k].co2_emission)
        sheet4.write(k+4, 10, pp[k].co2_cost)
        sheet4.write(k+4, 11, pp[k].bidprice)
        sheet4.write(k+4, 12, pp[k].bidvol)
        sheet4.write(k+4, 13, pp[k].profit)
        loc = pp[k].location
        xcood = loc[0]
        ycoord = loc[1]
        sheet4.write(k+4, 14, loc[0])
        sheet4.write(k+4, 15, loc[1])
        sheet4.write(k+4, 16, pp[k].tik)

        
    k_fin = len(pp)+6
    sheet4.write(k_fin, 0,'Decommissioned Power Plants')
    for kd in range(len(ppd)):
        sheet4.write(kd+k_fin, 1, ppd[kd].name)
        sheet4.write(kd+k_fin, 2, ppd[kd].source_type)
        sheet4.write(kd+k_fin, 3, ppd[kd].power)
        sheet4.write(kd+k_fin, 4, ppd[kd].power_day) 
        sheet4.write(kd+k_fin, 5, ppd[kd].health)
        sheet4.write(kd+k_fin, 6, ppd[kd].gen_cost)
        sheet4.write(kd+k_fin, 7, ppd[kd].cap_cost)
        sheet4.write(kd+k_fin, 8, ppd[kd].cur_val)        
        sheet4.write(kd+k_fin, 9, ppd[kd].co2_emission)
        sheet4.write(kd+k_fin, 10, ppd[kd].co2_cost)
        sheet4.write(kd+k_fin, 11, ppd[kd].bidprice)
        sheet4.write(kd+k_fin, 12, ppd[kd].bidvol)
        sheet4.write(kd+k_fin, 13, ppd[kd].profit)
        loc = ppd[kd].location
        xcood = loc[0]
        ycoord = loc[1]
        sheet4.write(kd+k_fin, 14, loc[0])
        sheet4.write(kd+k_fin, 15, loc[1])
        sheet4.write(kd+k_fin, 16, ppd[kd].tik)
        
        
        
        
        
    ### History of MCV and MCP ###
    sheet5 = data.add_worksheet("History of MCV and MCP Day {}".format(t))
    sheet5.write(0, 0, 'Day')
    sheet5.write(0, 1, 'MCV[MW]')
    sheet5.write(0, 2, 'MCP[€/MW]')
    for l in range(t):
        sheet5.write(l+1, 0, t)
        sheet5.write(l+1, 1, His_MCV[l])
        sheet5.write(l+1, 2, His_MCP[l])
    # plotting from the table(charts are independent of worksheets)
    chart1 = data.add_chart({'type': 'line'})
    chart1.set_y_axis({'name': 'MCV[MW]'})
    chart1.set_x_axis({'name': 'day'})
    chart1.set_title({'name': 'History of MCV'})
    data_start_loc1 = [1, 1] # xlsxwriter rquires list, no tuple
    data_end_loc1 = [data_start_loc1[0] + t, 1]
    chart1.add_series({
        'values': [sheet5.name] + data_start_loc1 + data_end_loc1,
        'name': "MCV",  
    })
    sheet5.insert_chart('D1', chart1)

    
    chart2 = data.add_chart({'type': 'line'})
    chart2.set_y_axis({'name': 'MCP[€/MW]'})
    chart2.set_x_axis({'name': 'day'})
    chart2.set_title({'name': 'History of MCP'})
    data_start_loc2 = [1, 2] # xlsxwriter rquires list, no tuple
    data_end_loc2 = [data_start_loc2[0] + t, 2]
    chart2.add_series({
        'values': [sheet5.name] + data_start_loc2 + data_end_loc2,
        'name': "MCP",        
    })
    sheet5.insert_chart('D17', chart2)
    
    
    
    
    
    
    
    ### History of fuel prices and CO2 cost ###
    sheet6 = data.add_worksheet("History of fuel and CO2 Day {}".format(t))
    sheet6.write(0, 0, 'Day')
    sheet6.write(0, 1, 'Uranium')
    sheet6.write(0, 2, 'Coal')
    sheet6.write(0, 3, 'Gas')
    sheet6.write(0, 4, 'CO2')
    for m in range(t):
        sheet6.write(m+1, 0, t)
        sheet6.write(m+1, 1, His_Fuel[m][0])
        sheet6.write(m+1, 2, His_Fuel[m][1])
        sheet6.write(m+1, 3, His_Fuel[m][2])
        sheet6.write(m+1, 4, His_Fuel[m][3])
        
    # plotting from the table(charts are independent of worksheets)  
    chart3 = data.add_chart({'type': 'line'})
    chart3.set_y_axis({'name': 'Price[€/ton]'})
    chart3.set_x_axis({'name': 'Day'})
    chart3.set_title({'name': 'History of fuel prices and CO2 emission cost'})
    
    data_start_loc3 = [1, 1] # xlsxwriter requires list, no tuple
    data_end_loc3 = [data_start_loc3[0] + t, 1]
    chart3.add_series({
        'values': [sheet6.name] + data_start_loc3 + data_end_loc3,
        'name': "Uranium",  
    })
    
    data_start_loc4 = [1, 2] # xlsxwriter requires list, no tuple
    data_end_loc4 = [data_start_loc4[0] + t, 2]
    chart3.add_series({
        'values': [sheet6.name] + data_start_loc4 + data_end_loc4,
        'name': "Coal",  
    })
    
    data_start_loc5 = [1, 3] # xlsxwriter requires list, no tuple
    data_end_loc5 = [data_start_loc5[0] + t, 3]
    chart3.add_series({
        'values': [sheet6.name] + data_start_loc5 + data_end_loc5,
        'name': "Gas",  
    })
    
    data_start_loc6 = [1, 4] # xlsxwriter requires list, no tuple
    data_end_loc6 = [data_start_loc6[0] + t, 4]
    chart3.add_series({
        'values': [sheet6.name] + data_start_loc6 + data_end_loc6,
        'name': "CO2 cost",  
    })
    sheet6.insert_chart('G1', chart3)   
    
    
    data.close() #write to file
    return

# Main Function

In [ ]:
##### Player Info ###
pname = str(input("Please enter your name: "))
pfundi = pfund # This records initial fund to calculate profit at the end of the game
pp = [] # List of active player power plants
pp_decom = [] # List of decomissioned player power plants
pp_all = [] # List of all player power plants

pp,pfund = buy_plant(pp,pfund,PPdata)

"""
### Player Info ###
pname = "Player"

pp1 = Source(name= "PP1-Nuclear", source='Nuclear', power=1000, gen_cost=0, cap_cost=900, co2_emission=0, co2_cost=0, location=[5,6])
pp2 = Source(name= "PP2-Coal", source='Coal', power=500, gen_cost=0, cap_cost=160, co2_emission=0, co2_cost=0, location=[3,1])
pp = [pp1,pp2]

"""

pp1 = Source(name= "PP1-Nuclear", source='Nuclear', power=1000, gen_cost=0, cap_cost=900, co2_emission=0, co2_cost=0, location=[5,6])
pp2 = Source(name= "PP2-Coal", source='Coal', power=500, gen_cost=0, cap_cost=160, co2_emission=0, co2_cost=0, location=[3,1])
pp_decom = [pp1,pp2]


# Create empty lists to save the history
His_MCP = []
His_MCV = []
His_Fuel = []


while (day<(dday+1)):
    
    print(("\n" + "\n" + "\n" + '\033[1m' + "Today's Date: " + str(day) + '\033[0m'), end='\n')
    
    fuel = fuel_price(day) # nuke,coal,gas
    
    AI_sorc_list, AI_sorc = dummy_source(NDummy_source,size_map,fuel,PPdata,AI_source_var)
    AI_sink_list, AI_sink = dummy_sink(NDummy_sink,size_map,AI_sink_var)
    
    update_fuel(pp,fuel)
    
    if choice1 == 1:
        update_renewable(pp,hydro_prob,wind_prob,solar_prob)
    else:
        weather_data(size_map,pp)
    
    ### Ask for Player Bids ###
    
    pbids = ask_bids(pp,day,fuel)
    
    ###########################
    
    sellbids_ai = sell_bids(AI_sorc,AI_sorc_list,AI_sell_bid_prof)
    buybids_ai = buy_bids(AI_sink,AI_sink_list,AI_buy_bid_impor)
    
    sellbids = sellbids_ai
    sellbids.extend(pbids)
    
    sellbids,buybids = merit_order(sellbids,buybids_ai)
    sellbids,buybids = cumulative_volume(sellbids,buybids)
    
    MCV,MCP = market(sellbids,buybids,day,choice2)
    
    check_bids(pp,MCV,MCP)
    
    # Export the history of Fuel and the trade information
    His_MCP.append(MCP)
    His_MCV.append(MCV)
    His_Fuel.append(fuel)
    
    pp,pp_decom = age_pp(pp,pp_decom,MCP,PPdata)
    pp,pfund = buy_plant(pp,pfund,PPdata)
    pp,pfund = sell_plant(pp,pfund)
    view_portfolio(pp,pp_decom,pfund)
    
    # For exporting the data
    exp_sellbids = sellbids
    exp_buybids = buybids
    
    if choice2 == 1:
        export_info(pname, pfund, day, AI_sorc, AI_sink, MCV, MCP, pp, pp_decom, His_MCV, His_MCP, His_Fuel, exp_sellbids, exp_buybids)
    
    if len(pp) == 0:
        break
    
    day = day + 1

pp_all = pp_all.extend(pp)
pp_all = pp_all.extend(pp_decom)

if pp_all is not None:
    for p in pp_all:
        pfund = pfund + p.profit
        p.profit = 0

print(("\n"+"\n" + "\n" + '\033[1m' + " *** GAME OVER *** " + '\033[0m'), end='\n')
print("Player Name : "+str(pname)+"\n")
print("Your net Bank Balance after this Playthrough : "+str('{:,.2f}'.format(pfund))+" €"+"\n")
print("Your Total Profit for this Playthrough : "+str('{:,.2f}'.format(pfund-pfundi))+" €"+"\n")

view_portfolio(pp,pp_decom,pfund)

In [ ]:
### TEST AREA ###

In [2]:
"""
#Requirements Files
%reload_ext watermark
%watermark -v -m -p numpy,matplotlib,random,math,xlwt,xlsxwriter,tabulate,watermark
"""

CPython 3.8.3
IPython 7.16.1

numpy 1.18.5
matplotlib 3.2.2
random unknown
math unknown
xlwt 1.3.0
xlsxwriter 1.2.9
tabulate 0.8.7
watermark 2.0.2

compiler   : MSC v.1916 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 158 Stepping 10, GenuineIntel
CPU cores  : 12
interpreter: 64bit
